# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [9]:
#download data from kaggle
import helper
import csv
import urllib.request
import zipfile  
import os

url = 'https://storage.googleapis.com/kaggle-datasets/203/430/the-simpsons-by-the-data.zip?GoogleAccessId=datasets@kaggle-161607.iam.gserviceaccount.com&Expires=1498385518&Signature=ctO9lXGSDIeJu%2FwA6%2BZiE9GVQAzdDzY3YnOQDLbAExM6HxA8SaythsHXgt87nbJuK6lGXQhLEhR1M%2BZEC2kMKTxH4KangihiN0LRo57E%2F7V6zAmX4F6KcfwCEA82aXBPDalGFCCOuOeyE%2BmL3ePJYBI3ViCAB2u0Zap7O8w%2FF16g3TcSu79%2B182p8ET4kGSztixkYoeL4MDsDcWSP65IQW3ZhNlzQgtVVu6Yp9MtsB0dpvSUY74j8C4kJktZPzm%2BJ243lYPPcI%2FFDo7cPZl%2FzxomqM5xsJICkC1gQhafUXHqvziawCHbnA%2B%2BLy4u9%2FNnoEggfgTRQfupIkMnNnT3lQ%3D%3D'
download_dir = './data/simpsons/data.zip'

if not os.path.isfile(download_dir):
    print('Start downloading, please wait a moment.\n')
    urllib.request.urlretrieve(url,download_dir)
    print('Successfully downloaded!\n')
    
# unzip file
    print('unzip data now\n')
    helper.un_zip('./data/simpsons/data.zip')
    print('Successfully unzipped!')
else:
    print('The data is already downloaded. Now loading it ')
    
# Load the full training data

data_dir = './data/simpsons/data.zip_files/simpsons_script_lines.csv'
text = helper.load_csv_data(data_dir)
print('Data loaded')

The data is already downloaded. Now loading it 
Load kaggle dataset
Data loaded


## Attention: If you want to use kaggle dataset, not to run the following 1 cell.

In [1]:
# """
# DON'T MODIFY ANYTHING IN THIS CELL
# """

# Load the udacity data
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data

Load Udacity dataset


In [2]:
text[:500]

"Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.\nBart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.\nMoe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?\nMoe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.\nMoe_Szyslak: What's the matter Homer? You're not your normal effervescent self.\nHomer_Simp"

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats


NameError: name 'text' is not defined

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = set(text)
    vocab_to_int = {word:ii for ii,word in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punc_to_token = {'.':'||Period||',
                    ',':'||Comma||',
                    '"':'||Quotation_Mark||',
                    ';':'||Semicolon||',
                    '!':'||Exclamation_mark||',
                    '?':'||Question_mark||',
                    '(':'||Left_Parentheses||',
                    ')':'||Right_Parentheses||',
                    '--':'||Dash||',
                    '\n':'||Return||'}
    return punc_to_token

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

Load Udacity dataset


# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [3]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

print('There are {:>6} unique words remain'.format(len(vocab_to_int)))

There are   6780 unique words remain


## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [5]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input_ = tf.placeholder(tf.int32,[None,None],name='input')
    target_ = tf.placeholder(tf.int32,[None,None],name='target')
    learning_rate = tf.placeholder(tf.float32,name='learning_rate')
    return input_,target_,learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [6]:
def get_init_cell(batch_size, rnn_size,num_layers):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
#     drop = tf.contrib.rnn.DropoutWrapper(lstm,output_keep_prob=0.5)
    cell = tf.contrib.rnn.MultiRNNCell([lstm for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state,name='initial_state')
    return (cell, initial_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [7]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embedding = tf.Variable(tf.random_uniform((vocab_size,embed_dim),-1,1))
    embed = tf.nn.embedding_lookup(embedding,input_data)
                        
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [8]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
#     outputs,final_state = tf.nn.dynamic_rnn(cell,inputs,initial_state=cell.zero_state(tf.placeholder(tf.int32,[],name='initial_state'), tf.float32))
    outputs,final_state = tf.nn.dynamic_rnn(cell,inputs,dtype=tf.float32)
    final_state = tf.identity(final_state,name='final_state')
    return (outputs,final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [9]:
def build_nn(cell,rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    inputs = get_embed(input_data,vocab_size,embed_dim)
    rnn,final_state = build_rnn(cell,inputs)
    logits = tf.contrib.layers.fully_connected(rnn,vocab_size)
    return logits,final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [10]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    num_batches = len(int_text)//(batch_size*seq_length)
    inputs = np.array(int_text[:num_batches*batch_size*seq_length+batch_size]).reshape(batch_size,-1)
    x,y = inputs[:,:-1],inputs[:,1:]
#     batches = np.zeros((num_batches,2,batch_size,seq_length),dtype=int)
    for n in range(0,num_batches):
#         batches[n,0] = x[:,n:n+seq_length]
#         batches[n,1] = y[:,n:n+seq_length]
#     print(batches[0,:,:1,:])
        yield x[:,n:n+seq_length],y[:,n:n+seq_length]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# tests.test_get_batches(get_batches)
# I use a generator here so the test can't work

"\nDON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE\n"

## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [14]:
# Number of Epochs
num_epochs = 500
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 256
# Number of RNN Layers
num_layers = 1
# Embedding Dimension Size
embed_dim = 300
# Sequence Length
seq_length = 15
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 5


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size,num_layers)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)
    

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    num_batches = len(int_text)//(batch_size*seq_length)

    for epoch_i in range(num_epochs):
        generator = get_batches(int_text, batch_size, seq_length)
        state = sess.run(initial_state, {input_text: next(generator)[0]})

        for ii, (x,y) in enumerate(generator):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * num_batches + ii) % show_every_n_batches == 0:
                print('Epoch {:>3}/{}  Batch {:>4}/{}   train_loss = {:.3f}'.format(                   
                    epoch_i,
                    num_epochs,
                    ii,
                    num_batches,
                    train_loss))
    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)


Epoch   0/500  Batch    0/8   train_loss = 8.822
Epoch   0/500  Batch    5/8   train_loss = 8.658
Epoch   1/500  Batch    2/8   train_loss = 8.234
Epoch   2/500  Batch    4/8   train_loss = 6.794
Epoch   3/500  Batch    1/8   train_loss = 6.390
Epoch   3/500  Batch    6/8   train_loss = 6.123
Epoch   4/500  Batch    3/8   train_loss = 5.989
Epoch   5/500  Batch    0/8   train_loss = 5.959
Epoch   5/500  Batch    5/8   train_loss = 5.925
Epoch   6/500  Batch    2/8   train_loss = 5.850
Epoch   7/500  Batch    4/8   train_loss = 5.804
Epoch   8/500  Batch    1/8   train_loss = 5.740
Epoch   8/500  Batch    6/8   train_loss = 5.775
Epoch   9/500  Batch    3/8   train_loss = 5.672
Epoch  10/500  Batch    0/8   train_loss = 5.649
Epoch  10/500  Batch    5/8   train_loss = 5.654
Epoch  11/500  Batch    2/8   train_loss = 5.569
Epoch  12/500  Batch    4/8   train_loss = 5.529
Epoch  13/500  Batch    1/8   train_loss = 5.470
Epoch  13/500  Batch    6/8   train_loss = 5.514
Epoch  14/500  Batch

Epoch 120/500  Batch    0/8   train_loss = 1.336
Epoch 120/500  Batch    5/8   train_loss = 1.471
Epoch 121/500  Batch    2/8   train_loss = 1.242
Epoch 122/500  Batch    4/8   train_loss = 1.331
Epoch 123/500  Batch    1/8   train_loss = 1.234
Epoch 123/500  Batch    6/8   train_loss = 1.584
Epoch 124/500  Batch    3/8   train_loss = 1.258
Epoch 125/500  Batch    0/8   train_loss = 1.288
Epoch 125/500  Batch    5/8   train_loss = 1.432
Epoch 126/500  Batch    2/8   train_loss = 1.204
Epoch 127/500  Batch    4/8   train_loss = 1.294
Epoch 128/500  Batch    1/8   train_loss = 1.194
Epoch 128/500  Batch    6/8   train_loss = 1.541
Epoch 129/500  Batch    3/8   train_loss = 1.224
Epoch 130/500  Batch    0/8   train_loss = 1.242
Epoch 130/500  Batch    5/8   train_loss = 1.397
Epoch 131/500  Batch    2/8   train_loss = 1.171
Epoch 132/500  Batch    4/8   train_loss = 1.265
Epoch 133/500  Batch    1/8   train_loss = 1.161
Epoch 133/500  Batch    6/8   train_loss = 1.504
Epoch 134/500  Batch

Epoch 240/500  Batch    0/8   train_loss = 0.952
Epoch 240/500  Batch    5/8   train_loss = 1.183
Epoch 241/500  Batch    2/8   train_loss = 0.980
Epoch 242/500  Batch    4/8   train_loss = 1.082
Epoch 243/500  Batch    1/8   train_loss = 0.957
Epoch 243/500  Batch    6/8   train_loss = 1.255
Epoch 244/500  Batch    3/8   train_loss = 1.031
Epoch 245/500  Batch    0/8   train_loss = 0.949
Epoch 245/500  Batch    5/8   train_loss = 1.181
Epoch 246/500  Batch    2/8   train_loss = 0.978
Epoch 247/500  Batch    4/8   train_loss = 1.081
Epoch 248/500  Batch    1/8   train_loss = 0.955
Epoch 248/500  Batch    6/8   train_loss = 1.252
Epoch 249/500  Batch    3/8   train_loss = 1.030
Epoch 250/500  Batch    0/8   train_loss = 0.947
Epoch 250/500  Batch    5/8   train_loss = 1.179
Epoch 251/500  Batch    2/8   train_loss = 0.976
Epoch 252/500  Batch    4/8   train_loss = 1.079
Epoch 253/500  Batch    1/8   train_loss = 0.954
Epoch 253/500  Batch    6/8   train_loss = 1.250
Epoch 254/500  Batch

KeyboardInterrupt: 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [47]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [56]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [44]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = tf.Graph.get_tensor_by_name(loaded_graph,'input:0')
    InitialStateTensor = tf.Graph.get_tensor_by_name(loaded_graph,'initial_state:0')
    FinalStateTensor = tf.Graph.get_tensor_by_name(loaded_graph,'final_state:0')
    ProbsTensor = tf.Graph.get_tensor_by_name(loaded_graph,'probs:0')
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [45]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    predict_word = int_to_vocab[np.argmax(probabilities)]
    return predict_word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [60]:
gen_length = 500
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'barney_gumble'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

barney_gumble:(drunk) look, the thing is my god. it's you, and god bless america.

homer_simpson:(disappointed) it's over.
carl_carlson: oh, i'd love to discuss your problems.
lenny_leonard: hey, homer. i don't need to learn / 'cause we're going to a secret with the.
carl_carlson: that's the this. / canyonero(then) ahh. what the monster! / yeah, we're going to the top.(frink noise)
lisa_simpson:(sighs) why did i ever think i needed a clone?
moe_szyslak: what's the matter, homer? my militia has a secret christian underpants one leg at a time.
moe_szyslak:(")
homer_simpson:(" shoo fly shoo") calf's in the field so you and god bless america.


lenny_leonard: it's not an apology for, the treasure on the eyes.
moe_szyslak:(still) i don't want to the pool table.




carl_carlson: i'd say(throws another punch) i know! that's the beauty people. i'm need a little more like! i had the new no.
lenny_leonard: no...
carl_carlson: ah, no.
lenny_leonard: yeah, nice try there, twelve-step.


homer_sim

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.